In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
! unzip  /content/drive/MyDrive/released.zip -d ./data


Archive:  /content/drive/MyDrive/released.zip
  inflating: ./data/test/00cb0c05-992f-4b41-83ed-842e4f5239ba.pkl  
  inflating: ./data/test/01cf3373-4311-4806-8b4e-44473e91328b.pkl  
  inflating: ./data/test/03678718-0a78-4967-8520-6cb16ec03173.pkl  
  inflating: ./data/test/0a3d2366-2c62-489a-a9ec-069dbdfc07f3.pkl  
  inflating: ./data/test/0d7939d5-a7fb-4f06-a447-8b29657c2865.pkl  
  inflating: ./data/test/1140ab06-9452-4fb8-a82d-9285a37d165a.pkl  
  inflating: ./data/test/12b83c8b-c899-4870-92d0-8448f07fe111.pkl  
  inflating: ./data/test/168d8258-49fc-42d4-bf46-334eb243c571.pkl  
  inflating: ./data/test/16a7df27-aa1f-4ef9-b62e-3fb7762cf60d.pkl  
  inflating: ./data/test/175058f1-d0c1-4333-b907-014d97d0105a.pkl  
  inflating: ./data/test/17aa595a-1ed8-4e47-9632-032b4b27ded3.pkl  
  inflating: ./data/test/17c35bb6-59e8-4799-aa91-c9dd4b018a82.pkl  
  inflating: ./data/test/1cff53df-66ea-428a-ac43-5b7c5f0b1a7f.pkl  
  inflating: ./data/test/1d1f4e00-d4b0-4701-9683-4421af511b2b.pkl  
  

In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import pickle
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

class BagDataset(Dataset):
    def __init__(self, bags, labels=None, transform=None):
        self.bags = bags
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.bags)

    def __getitem__(self, idx):
        bag = self.bags[idx]
        if self.transform:
            bag = [self.transform(Image.fromarray(image)) for image in bag]
        bag = torch.stack(bag)
        if self.labels is not None:
            label = self.labels[idx]
            return bag, label
        else:
            return bag

from torchvision.models import ResNet50_Weights

class SimpleClassifier(nn.Module):
    def __init__(self):
        super(SimpleClassifier, self).__init__()
        self.resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.mean(x, dim=1)
        x = self.resnet(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

def load_data(train_path, test_path):
    train_bags = []
    train_labels = []
    test_bags = []
    test_ids = []

    for class_label in [0, 1]:
        class_path = os.path.join(train_path, f'class_{class_label}')
        for file_name in os.listdir(class_path):
            with open(os.path.join(class_path, file_name), 'rb') as f:
                bag = pickle.load(f)
                train_bags.append(bag)
                train_labels.append(class_label)

    for file_name in os.listdir(test_path):
        with open(os.path.join(test_path, file_name), 'rb') as f:
            bag = pickle.load(f)
            test_bags.append(bag)
            test_ids.append(file_name.split('.')[0])

    return train_bags, train_labels, test_bags, test_ids

def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=50):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = outputs > 0.5

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        scheduler.step()

    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

if __name__ == '__main__':
    data_dir = './data'
    train_path = os.path.join(data_dir, 'train')
    test_path = os.path.join(data_dir, 'test')

    train_bags, train_labels, test_bags, test_ids = load_data(train_path, test_path)

    transform = transforms.Compose([
        transforms.RandomResizedCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_dataset = BagDataset(train_bags, train_labels, transform=transform)

    val_size = int(0.2 * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

    dataloaders = {'train': train_loader, 'val': val_loader}
    dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = SimpleClassifier().to(device)

    class_weights = compute_class_weight('balanced', classes=[0, 1], y=train_labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = nn.BCELoss()

    optimizer = optim.Adam(model.parameters(), lr=1e-6)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    model = train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=100)

    model_path = '/content/drive/MyDrive/trained_classifier.pth'
    torch.save(model.state_dict(), model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 85.1MB/s]


Epoch 0/99
----------
train Loss: 0.7150 Acc: 0.4792
val Loss: 0.6832 Acc: 0.5500
Epoch 1/99
----------
train Loss: 0.6954 Acc: 0.5167
val Loss: 0.7042 Acc: 0.4667
Epoch 2/99
----------
train Loss: 0.6998 Acc: 0.5000
val Loss: 0.6947 Acc: 0.5333
Epoch 3/99
----------
train Loss: 0.7063 Acc: 0.4500
val Loss: 0.6857 Acc: 0.5833
Epoch 4/99
----------
